In [2]:
# Chatbot usando TensorFlow e NLTK 
import json
import numpy as np
import random
import nltk
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [3]:
# Reprodutibilidade
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
# NLTK 
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
lemmatizer = WordNetLemmatizer()
ignore_words = ['?', '!', '.', ',', '´', '`', '``', "''"]

In [5]:
#dataset
intents = {
    "intents": [
        {"tag": "troca", "patterns": ["Quero trocar o produto", "Meu produto veio com defeito", "Posso devolver?"]},
        {"tag": "garantia", "patterns": ["Qual o tempo de garantia?", "Minha garantia venceu?", "O que cobre a garantia?", 
                                         "Como funciona a garanta?"]},
        {"tag": "pedido", "patterns": ["Qual o status do meu pedido?", "Quer acompanhar a entrega", "Onde está minha compra?", 
                                       "Rastrear pedido"]},
        {"tag": "saudacao", "patterns": ["Oi", "Olá", "Boa tarde", "Bom dia"]}
    ]
}

respostas = {
    "troca": "Claro! Por favor, envie o número do seu pedido para prosseguirmos com a troca.",
    "garantia": "Nossa garantia é de 12 meses. Posso verificar o status da sua cobertra.",
    "pedido": "Você pode acompanhar seu pedido neste link: ...",
    "saudacao": "Olá! Seja bem-vindo ao nosso atendimento. Como posso ajudar?"
}


In [6]:
#Tokenização e lematização
def tokenize_and_lemmatize(text: str):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [t for t in tokens if t not in ignore_words]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens

words, classes, documents = [], [], []
for intent in intents['intents']:
    tag = intent['tag']
    if tag not in classes:
        classes.append(tag)
    for pattern in intent['patterns']:
        toks = tokenize_and_lemmatize(pattern)
        words.extend(toks)
        documents.append((toks, tag))

words = sorted(list(set(words)))
classes = sorted(classes)

In [7]:
#Vetorização
def vetorizar(frase: str):
    toks = tokenize_and_lemmatize(frase)
    bag = [1 if w in toks else 0 for w in words]
    return np.array(bag, dtype=int)



In [8]:
#Cosntruindo o modelo
train_x, train_y = [], []
for doc in documents:
    bag = [1 if w in doc[0] else 0 for w in words]
    train_x.append(bag)
    train_y.append(classes.index(doc[1]))

train_x = np.array(train_x)
train_y = np.array(train_y)

model_tf = Sequential([
    Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(classes), activation='softmax')
])
model_tf.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])
print("Treinando modelo...")
model_tf.fit(train_x, train_y, epochs=200, batch_size=8, verbose=0)
print("Modelo concluído.")

Treinando modelo...
Modelo concluído.


In [9]:

def classificar_intencao(frase: str):
    bag = vetorizar(frase)
    probs = model_tf.predict(np.array([bag]), verbose=0)[0]
    idx = int(np.argmax(probs))
    return classes[idx], float(probs[idx])

In [10]:
def responder(frase: str):
    tag, conf = classificar_intencao(frase)
    resp = respostas.get(tag, "Desculpe, não entendi sua solicitação.")
    return resp, tag, conf


In [12]:
testes = [
    "Oi",
    "Quero trocar meu produto com defeito",
    "Qual a garanta?",
    "Como está meu pedido?",
    "Meu aparelho está com defeito e quero devolver",
    "Olá preciso conserto"
]

print("Chatbot-prototipo — simulação de teste\n")
for entrada in testes:
    resposta_texto, tag, conf = responder(entrada)
    print(f"Você: {entrada}")
    print(f"Bot: {resposta_texto}")
    print(f"  -> intenção: {tag}, confiança: {conf:.2f}\n")

Chatbot prototipo — simulação de teste

Você: Oi
Bot: Olá! Seja bem-vindo ao nosso atendimento. Como posso ajudar?
  -> intenção: saudacao, confiança: 1.00

Você: Quero trocar meu produto com defeito
Bot: Claro! Por favor, envie o número do seu pedido para prosseguirmos com a troca.
  -> intenção: troca, confiança: 1.00

Você: Qual a garanta?
Bot: Nossa garantia é de 12 meses. Posso verificar o status da sua cobertra.
  -> intenção: garantia, confiança: 1.00

Você: Como está meu pedido?
Bot: Você pode acompanhar seu pedido neste link: ...
  -> intenção: pedido, confiança: 1.00

Você: Meu aparelho está com defeito e quero devolver
Bot: Claro! Por favor, envie o número do seu pedido para prosseguirmos com a troca.
  -> intenção: troca, confiança: 1.00

Você: Olá preciso conserto
Bot: Olá! Seja bem-vindo ao nosso atendimento. Como posso ajudar?
  -> intenção: saudacao, confiança: 1.00

